Course Human-Centered Data Science ([HCDS](https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_human_centered_data_science.html)) - Winter Term 2020/21 - [HCC](https://www.mi.fu-berlin.de/en/inf/groups/hcc/index.html) | [Freie Universität Berlin](https://www.fu-berlin.de/)

***

# A2 - Wikipedia, ORES, and Bias in Data
Please follow the reproducability workflow as practiced during the last exercise.

## Step 1⃣ | Data acquisition

You will use two data sources: (1) Wikipedia articles of politicians and (2) world population data.

**Wikipedia articles -**
The Wikipedia articles can be found on [Figshare](https://figshare.com/articles/Untitled_Item/5513449). It contains politiciaans by country from the English-language wikipedia. Please read through the documentation for this repository, then download and unzip it to extract the data file, which is called `page_data.csv`.

**Population data -**
The population data is available in `CSV` format in the `_data` folder. The file is named `export_2019.csv`. This dataset is drawn from the [world population datasheet](https://www.prb.org/international/indicator/population/table/) published by the Population Reference Bureau (downloaded 2020-11-13 10:14 AM). I have edited the dataset to make it easier to use in this assignment. The population per country is given in millions!

### Import Libraries

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

### Create DataFrames for both CSV in src

In [34]:
path_cwd = os.getcwd()
path_cwd

'/home/anil/.git/hcds-winter-2020/assignments/A3_Bias/ansa'

In [35]:
path_data_raw = os.path.join(path_cwd, 'data_raw')
path_data_raw

'/home/anil/.git/hcds-winter-2020/assignments/A3_Bias/ansa/data_raw'

In [36]:
path_file = os.path.join(path_data_raw, 'page_data.csv')
articles_politicians = pd.read_csv(path_file)
articles_politicians.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


In [37]:
path_file = os.path.join(path_data_raw, 'export_2019.csv')
articles_population = pd.read_csv(path_file, sep=';')
articles_population.head()

,country,population,region
0,Algeria,44.357,AFRICA
1,Egypt,100.803,AFRICA
2,Libya,6.891,AFRICA
3,Morocco,35.952,AFRICA
4,Sudan,43.849,AFRICA


### Get some insight about the data

In [38]:
articles_politicians.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47197 entries, 0 to 47196
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   page     47197 non-null  object
 1   country  47197 non-null  object
 2   rev_id   47197 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [39]:
articles_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     210 non-null    object 
 1   population  210 non-null    float64
 2   region      210 non-null    object 
dtypes: float64(1), object(2)
memory usage: 5.0+ KB


## Step 2⃣ | Data processing and cleaning
The data in `page_data.csv` contain some rows that you will need to filter out. It contains some page names that start with the string `"Template:"`. These pages are not Wikipedia articles, and should not be included in your analysis. The data in `export_2019.csv` does not need any cleaning.

***

| | `page_data.csv` | | |
|-|------|---------|--------|
| | **page** | **country** | **rev_id** |
|0|	Template:ZambiaProvincialMinisters | Zambia | 235107991 |
|1|	Bir I of Kanem | Chad | 355319463 |

***

| | `export_2019.csv` | | |
|-|------|---------|--------|
| | **country** | **population** | **region** |
|0|	Algeria | 44.357 | AFRICA |
|1|	Egypt | 100.803 | 355319463 |

***

### Filter out Templates

In [40]:
articles_politicians.shape

(47197, 3)

In [41]:
# Filter
articles_politicians = articles_politicians[articles_politicians.page.str.contains('Template:', na=False) == False]
articles_politicians.shape

(46701, 3)

In [42]:
articles_politicians.head()

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568


### Save the clean data 

In [43]:
path_data_clean = os.path.join(path_cwd, 'data_clean')
path_data_clean

'/home/anil/.git/hcds-winter-2020/assignments/A3_Bias/ansa/data_clean'

In [44]:
path_file = os.path.join(path_data_clean, 'clean_page_data.csv')
articles_politicians.to_csv(path_file, index = False)

### Getting article quality predictions with ORES

Now you need to get the predicted quality scores for each article in the Wikipedia dataset. We're using a machine learning system called [**ORES**](https://www.mediawiki.org/wiki/ORES) ("Objective Revision Evaluation Service"). ORES estimates the quality of an article (at a particular point in time), and assigns a series of probabilities that the article is in one of the six quality categories. The options are, from best to worst:

| ID | Quality Category |  Explanation |
|----|------------------|----------|
| 1 | FA    | Featured article |
| 2 | GA    | Good article |
| 3 | B     | B-class article |
| 4 | C     | C-class article |
| 5 | Start | Start-class article |
| 6 | Stub  | Stub-class article |

For context, these quality classes are a sub-set of quality assessment categories developed by Wikipedia editors. If you're curious, you can [read more](https://en.wikipedia.org/wiki/Wikipedia:Content_assessment#Grades) about what these assessment classes mean on English Wikipedia. For this assignment, you only need to know that these categories exist, and that ORES will assign one of these six categories to any `rev_id`. You need to extract all `rev_id`s in the `page_data.csv` file and use the ORES API to get the predicted quality score for that specific article revision.

### ORES REST API endpoint

The [ORES REST API](https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context_revid_model) is configured fairly similarly to the pageviews API we used for the last assignment. It expects the following parameters:

* **project** --> `enwiki`
* **revid** --> e.g. `235107991` or multiple ids e.g.: `235107991|355319463` (batch)
* **model** --> `wp10` - The name of a model to use when scoring.

**❗Note on batch processing:** Please read the documentation about [API usage](https://www.mediawiki.org/wiki/ORES#API_usage) if you want to query a large number of revisions (batches). 

You will notice that ORES returns a prediction value that contains the name of one category (e.g. `Start`), as well as probability values for each of the six quality categories. For this assignment, you only need to capture and use the value for prediction.

**❗Note:** It's possible that you will be unable to get a score for a particular article. If that happens, make sure to maintain a log of articles for which you were not able to retrieve an ORES score. This log should be saved as a separate file named `ORES_no_scores.csv` and should include the `page`, `country`, and `rev_id` (just as in `page_data.csv`).

You can use the following **samle code for API calls**:

In [45]:
import requests
import json

# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/AnilSahintuerk',
    'From': 'ilkas96@zedat.fu-berlin.de'
}

def get_ores_data(rev_id, headers):
    
    # Define the endpoint
    # https://ores.wikimedia.org/scores/enwiki/?models=wp10&revids=807420979|807422778
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    params = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : rev_id
              }

    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    data = json.dumps(response)

    return data

Sending one request for each `rev_id` might take some time. If you want to send batches you can use `'|'.join(str(x) for x in revision_ids` to put your ids together. Please make sure to deal with [exception handling](https://www.w3schools.com/python/python_try_except.asp) of the `KeyError` exception, when extracting the `prediction` from the `JSON` response.

### Creating ORES DataFrame

In [46]:
rev_id = [x for x in articles_politicians['rev_id']]

#### Making the requests

In [47]:
predictions = []
ores_no_score = []

for i in range(0, len(rev_id) // 50+1):

    batch = '|'.join(str(x) for x in rev_id[(i * 50):(50 * (i+1))])
    data = get_ores_data(batch, headers)
    data = json.loads(data)
    
    for key in data['enwiki']['scores'].keys():
        try:
            prediction = data['enwiki']['scores'][str(key)]['wp10']['score']['prediction']
            predictions.append([key, prediction])
        except KeyError:
            ores_no_score.append(key)

assert len(rev_id) == len(predictions) + len(ores_no_score), 'Not working properly'

In [48]:
ores_no_score_df = pd.DataFrame(ores_no_score, columns=['rev_id'])
ores_no_score_df['rev_id'] = ores_no_score_df['rev_id'].astype(int)
ores_no_score_df = pd.merge(ores_no_score_df, articles_politicians, on='rev_id', how='left')


assert int(ores_no_score[2]) == ores_no_score_df.iloc[2]['rev_id']
assert int(ores_no_score[12]) == ores_no_score_df.iloc[12]['rev_id']
assert len(ores_no_score) == ores_no_score_df.shape[0]

ores_no_score_df.head()

,rev_id,page,country
0,516633096,List of politicians in Poland,Poland
1,550682925,Tingtingru,Vanuatu
2,627547024,Daud Arsala,Afghanistan
3,671484594,Bharat Saud,Nepal
4,684023803,Robert Sych,Poland


In [49]:
quality_df = pd.DataFrame(predictions, columns=['rev_id', 'article_quality'])
quality_df['rev_id'] = quality_df['rev_id'].astype(int)
quality_df.head()

,rev_id,article_quality
0,355319463,Stub
1,393276188,Stub
2,393822005,Stub
3,395521877,Stub
4,395526568,Stub


### Save new DataFrames

In [50]:
path_file = os.path.join(path_data_clean, 'quality.csv')
quality_df.to_csv(path_file, index = False)    

In [51]:
path_file = os.path.join(path_data_clean, 'ORES_no_scores.csv')
ores_no_score_df.to_csv(path_file, index = False) 

### Combining the datasets

Now you need to combine both dataset: (1) the wikipedia articles and its ORES quality scores and (2) the population data. Both have columns named `country`. After merging the data, you'll invariably run into entries which cannot be merged. Either the population dataset does not have an entry for the equivalent Wikipedia country, or vis versa.

Please remove any rows that do not have matching data, and output them to a `CSV` file called `countries-no_match.csv`. Consolidate the remaining data into a single `CSV` file called `politicians_by_country.csv`.

The schema for that file should look like the following table:


| article_name | country | region | revision_id | article_quality | population |
|--------------|---------|--------|-------------|-----------------|------------|
| Bir I of Kanem | Chad  | AFRICA | 807422778 | Stub | 16877000 |

### Fix some Typos

In [52]:
articles_population.loc[articles_population['country'] == 'Czechia', 'country'] = 'Czech Republic'
articles_population.loc[articles_population['country'] == 'Czech Republic']

,country,population,region
170,Czech Republic,10.716,EUROPE


In [53]:
ores_df = pd.merge(articles_politicians, articles_population, on='country', how='left')

assert ores_df.shape[0] == articles_politicians.shape[0]
ores_df.head()

,page,country,rev_id,population,region
0,Bir I of Kanem,Chad,355319463,16.877,AFRICA
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,5.008,ASIA
2,Yos Por,Cambodia,393822005,15.497,ASIA
3,Julius Gregr,Czech Republic,395521877,10.716,EUROPE
4,Edvard Gregr,Czech Republic,395526568,10.716,EUROPE


In [54]:
ores_df = pd.merge(ores_df, quality_df, on='rev_id', how='left')
ores_df.head()

,page,country,rev_id,population,region,article_quality
0,Bir I of Kanem,Chad,355319463,16.877,AFRICA,Stub
1,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188,5.008,ASIA,Stub
2,Yos Por,Cambodia,393822005,15.497,ASIA,Stub
3,Julius Gregr,Czech Republic,395521877,10.716,EUROPE,Stub
4,Edvard Gregr,Czech Republic,395526568,10.716,EUROPE,Stub


### The NaN in article quality must be equal to the length of ores_no_score

In [55]:
ores_df['article_quality'].isna().sum()

223

In [56]:
len(ores_no_score)

223

### Filter the rows out with article_quality being NaN

In [57]:
ores_df = ores_df[ores_df['article_quality'].notna()]

In [58]:
ores_df['article_quality'].isna().sum()

0

In [59]:
assert ores_df.shape[0] == len(predictions), 'ores_df needs to have the same row number like entries in predictions'

In [60]:
ores_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46478 entries, 0 to 46699
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   page             46478 non-null  object 
 1   country          46478 non-null  object 
 2   rev_id           46478 non-null  int64  
 3   population       44872 non-null  float64
 4   region           44872 non-null  object 
 5   article_quality  46478 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.5+ MB


### Handle row where Population is NaN

In [61]:
path_results = os.path.join(path_cwd, 'results')
path_file = os.path.join(path_results, 'no_population.csv')
no_population_df = ores_df[ores_df.population.isna()].groupby('country').sum()
no_population_df.to_csv(path_file)

In [62]:
ores_df = ores_df[ores_df['population'].notna()]

In [63]:
ores_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44872 entries, 0 to 46699
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   page             44872 non-null  object 
 1   country          44872 non-null  object 
 2   rev_id           44872 non-null  int64  
 3   population       44872 non-null  float64
 4   region           44872 non-null  object 
 5   article_quality  44872 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.4+ MB


### Save ores_df

In [64]:
path_file = os.path.join(path_results, 'ores_df.csv')
ores_df.to_csv(path_file, index = False)

## Step 3⃣ | Analysis

Your analysis will consist of calculating the proportion (as a percentage) of articles-per-population (we can also call it `coverage`) and high-quality articles (we can also call it `relative-quality`)for **each country** and for **each region**. By `"high quality"` arcticle we mean an article that ORES predicted as `FA` (featured article) or `GA` (good article).

**Examples:**

* if a country has a population of `10,000` people, and you found `10` articles about politicians from that country, then the percentage of `articles-per-population` would be `0.1%`.
* if a country has `10` articles about politicians, and `2` of them are `FA` or `GA` class articles, then the percentage of `high-quality-articles` would be `20%`.

### Results format

The results from this analysis are six `data tables`. Embed these tables in the Jupyter notebook. You do not need to graph or otherwise visualize the data for this assignment. The tables will show:

1. **Top 10 countries by coverage**<br>10 highest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Bottom 10 countries by coverage**<br>10 lowest-ranked countries in terms of number of politician articles as a proportion of country population
1. **Top 10 countries by relative quality**<br>10 highest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Bottom 10 countries by relative quality**<br>10 lowest-ranked countries in terms of the relative proportion of politician articles that are of GA and FA-quality
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the total count of politician articles from countries in each region as a proportion of total regional population
1. **Regions by coverage**<br>Ranking of regions (in descending order) in terms of the relative proportion of politician articles from countries in each region that are of GA and FA-quality

**❗Hint:** You will find what country belongs to which region (e.g. `ASIA`) also in `export_2019.csv`. You need to calculate the total poulation per region. For that you could use `groupby` and also check out `apply`.

## Creating sub DataFrames from ores_df

Creating sub Dataframes because we have to use 'groupby' and working with a df.groupby.object is a little harder than a df.object. That's why I choose to make different DataFrames and merge these Subdataframes together. For every sub Datafram there is a certain order of steps.

1. copy ores_df
2. delete columns I dont need
3. add the certain 'groupby' attribute
4. Rename cols if needed

### Create Population Dataframe

In [65]:
# DF with only Country, Population and Region
population_df = ores_df.drop(['page', 'rev_id', 'article_quality'], axis=1)
population_df = population_df.drop_duplicates(keep='first')

# Add total Region Population
populations = population_df.groupby('region').sum()
population_df = pd.merge(population_df, populations, how='right', on='region')

population_df = population_df.rename(columns={'population_x':'population_country', 
                                              'population_y':'population_region'})
# Create proportion attribute
# proportion := country population percantage of region population
population_df['proportion'] = population_df['population_country'] / population_df['population_region']
population_df.head()

,country,population_country,region,population_region,proportion
0,Chad,16.877,AFRICA,2859.681,0.005902
1,Egypt,100.803,AFRICA,2859.681,0.035250
2,Malawi,19.130,AFRICA,2859.681,0.006690
3,Ghana,31.073,AFRICA,2859.681,0.010866
4,Tanzania,59.734,AFRICA,2859.681,0.020888


### Create Coverage Df

page = the number of articles in the Dataset for the country
proportion = page / population_country

In [66]:
coverage_df = ores_df.groupby('country', as_index=False).count()
coverage_df = coverage_df.drop(['region', 'rev_id', 'article_quality', 'population'], axis=1)
coverage_df = pd.merge(coverage_df, population_df, on='country', how='inner')
coverage_df = coverage_df.drop(['region', 'population_region', 'proportion'], axis=1)
coverage_df['proportion'] = coverage_df['page'] / coverage_df['population_country']
coverage_df

,country,page,population_country,proportion
0,Afghanistan,320,38.928,8.220304
1,Albania,457,2.838,161.028894
2,Algeria,116,44.357,2.615145
3,Andorra,34,82.000,0.414634
4,Angola,106,32.522,3.259332
...,...,...,...,...
179,Venezuela,131,28.645,4.573224
180,Vietnam,187,96.209,1.943685
181,Yemen,117,29.826,3.922752
182,Zambia,25,18.384,1.359878


### Create Quality DF

In [67]:
quality_df = ores_df.copy()
quality_df = quality_df.drop(['rev_id', 'page', 'population'], axis=1)
quality_df

,country,region,article_quality
0,Chad,AFRICA,Stub
1,Palestinian Territory,ASIA,Stub
2,Cambodia,ASIA,Stub
3,Czech Republic,EUROPE,Stub
4,Czech Republic,EUROPE,Stub
...,...,...,...
46695,United States,NORTHERN AMERICA,C
46696,Gambia,AFRICA,GA
46697,United States,NORTHERN AMERICA,C
46698,Saudi Arabia,ASIA,GA


### Create relative Quality DF

In [68]:
relative_quality_df = quality_df.copy()
relative_quality_df = pd.merge(relative_quality_df, coverage_df, on='country', how='left')
relative_quality_df = relative_quality_df.loc[(relative_quality_df.article_quality =='GA') 
                                              | (relative_quality_df.article_quality =='FA')]

total_qualtiy_df = relative_quality_df.groupby('country').count()
total_qualtiy_df = total_qualtiy_df.drop(['page', 'proportion', 'population_country', 'region'], axis=1)
total_qualtiy_df = total_qualtiy_df.rename(columns={'article_quality':'total_quality'})

relative_quality_df = pd.merge(coverage_df, total_qualtiy_df,
                               on='country', how='inner')

relative_quality_df['article_proportion'] = relative_quality_df['total_quality'] / relative_quality_df['proportion']
relative_quality_df = relative_quality_df.drop(['page', 'proportion', 'population_country'], axis=1)

relative_quality_df

,country,total_quality,article_proportion
0,Afghanistan,13,1.581450
1,Albania,3,0.018630
2,Algeria,2,0.764776
3,Argentina,16,1.478680
4,Armenia,5,0.076186
...,...,...,...
142,Vanuatu,3,16.603448
143,Venezuela,3,0.655992
144,Vietnam,13,6.688326
145,Yemen,3,0.764769


### 1. Top 10 countries by coverage

In [69]:
top_countries_df = coverage_df.sort_values(by='proportion', ascending=False)[0:10]
top_countries_df = top_countries_df.reset_index()
top_countries_df.index += 1
top_countries_df = top_countries_df.drop('index', axis=1)
top_countries_df

,country,page,population_country,proportion
1,Tuvalu,54,0.010,5400.000000
2,Albania,457,2.838,161.028894
3,New Zealand,783,4.987,157.008221
4,Norway,656,5.387,121.774643
5,Moldova,423,3.535,119.660537
6,Estonia,148,1.331,111.194591
7,Finland,569,5.529,102.911919
8,Sao Tome and Principe,21,0.210,100.000000
9,Lithuania,244,2.794,87.329993
10,Uruguay,285,3.531,80.713679


In [70]:
# save csv
path_file = os.path.join(path_results, 'top_countries.csv')
top_countries_df.to_csv(path_file)

### 2. Bottom 10 countries by coverage

In [71]:
bottom_countries_df = coverage_df.sort_values(by='proportion')[0:10]
bottom_countries_df = bottom_countries_df.reset_index()
bottom_countries_df.index += 1
bottom_countries_df = bottom_countries_df.drop('index', axis=1)
bottom_countries_df

,country,page,population_country,proportion
1,Guyana,20,787.0,0.025413
2,Djibouti,37,988.0,0.037449
3,Belize,16,419.0,0.038186
4,Barbados,14,287.0,0.048780
5,Bahamas,20,393.0,0.050891
6,Cape Verde,36,556.0,0.064748
7,Suriname,40,605.0,0.066116
8,French Guiana,27,294.0,0.091837
9,Martinique,34,356.0,0.095506
10,Montenegro,72,622.0,0.115756


In [72]:
# save csv
path_file = os.path.join(path_results, 'bottom_countries.csv')
bottom_countries_df.to_csv(path_file)

### 3. Top 10 countries by relative quality

In [73]:
top_relative_quality_df = relative_quality_df.sort_values(by='article_proportion', ascending=False)[0:10]
top_relative_quality_df = top_relative_quality_df.reset_index()
top_relative_quality_df.index += 1
top_relative_quality_df = top_relative_quality_df.drop('index', axis=1)

top_relative_quality_df

,country,total_quality,article_proportion
1,China,40,49.685917
2,United States,80,24.686848
3,India,13,18.706372
4,Montenegro,2,17.277778
5,Vanuatu,3,16.603448
6,Suriname,1,15.125000
7,Indonesia,9,11.701679
8,Martinique,1,10.470588
9,Vietnam,13,6.688326
10,Maldives,1,6.518072


In [74]:
# save csv
path_file = os.path.join(path_results, 'top_relative_quality.csv')
top_relative_quality_df.to_csv(path_file)

### 4. Bottom 10 countries by relative quality

In [75]:
bottom_relative_quality_df = relative_quality_df.sort_values(by='article_proportion')[0:10]
bottom_relative_quality_df = bottom_relative_quality_df.reset_index()
bottom_relative_quality_df.index += 1
bottom_relative_quality_df = bottom_relative_quality_df.drop('index', axis=1)

bottom_relative_quality_df

,country,total_quality,article_proportion
1,Tuvalu,4,0.000741
2,Lithuania,1,0.011451
3,Cyprus,1,0.012443
4,Albania,3,0.018630
5,Mauritius,1,0.019462
6,Switzerland,1,0.021478
7,Belgium,1,0.022187
8,Uruguay,2,0.024779
9,Portugal,1,0.032047
10,Latvia,1,0.033964


In [76]:
# save csv
path_file = os.path.join(path_results, 'bottom_relative_quality.csv')
bottom_relative_quality_df.to_csv(path_file)

### 5. Regions Coverage

In [77]:
regions_df = pd.merge(population_df, coverage_df, on='country', how='inner')
regions_df = regions_df.drop(['population_country_y', 'proportion_y', 'proportion_x'], axis=1)

regions_df = regions_df.groupby('region', as_index=False).sum()
regions_df['proportion'] = regions_df['page'] / regions_df['population_region'] 
regions_df = regions_df.sort_values(by='proportion', ascending=False)
regions_df = regions_df.reset_index()
regions_df.index += 1
regions_df = regions_df.drop('index', axis=1)


regions_df

,region,population_country_x,population_region,page,proportion
1,NORTHERN AMERICA,368.068,736.136,1910,2.594629
2,EUROPE,3078.860,132390.980,16027,0.121058
3,OCEANIA,2369.701,28436.412,3127,0.109965
4,ASIA,5147.988,231659.460,11691,0.050466
5,AFRICA,2859.681,145843.731,6844,0.046927
6,LATIN AMERICA AND THE CARIBBEAN,4425.712,137197.072,5273,0.038434


In [78]:
# save csv
path_file = os.path.join(path_results, 'regions_articles.csv')
regions_df.to_csv(path_file)

### 6. Regions Coverage

In [79]:
regions_df = pd.merge(population_df, relative_quality_df, on='country', how='inner')

regions_df = regions_df.groupby('region', as_index=False).sum()
regions_df['proportion'] = regions_df['total_quality'] / regions_df['population_region'] 
regions_df = regions_df.drop(['population_country', 'article_proportion'], axis=1)
regions_df = regions_df.sort_values(by='proportion', ascending=False)
regions_df = regions_df.reset_index()
regions_df.index += 1
regions_df = regions_df.drop('index', axis=1)

regions_df

,region,population_region,proportion,total_quality
1,NORTHERN AMERICA,736.136,0.141278,104
2,OCEANIA,14218.206,0.004431,63
3,EUROPE,110838.960,0.003167,351
4,ASIA,216215.496,0.001462,316
5,AFRICA,111527.559,0.001067,119
6,LATIN AMERICA AND THE CARIBBEAN,97365.664,0.000781,76


In [80]:
# save csv
path_file = os.path.join(path_results, 'regions_quality.csv')
regions_df.to_csv(path_file)

***

#### Credits

This exercise is slighty adapted from the course [Human Centered Data Science (Fall 2019)](https://wiki.communitydata.science/Human_Centered_Data_Science_(Fall_2019)) of [Univeristy of Washington](https://www.washington.edu/datasciencemasters/) by [Jonathan T. Morgan](https://wiki.communitydata.science/User:Jtmorgan).

Same as the original inventors, we release the notebooks under the [Creative Commons Attribution license (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/).